In [ ]:
from config import *
from data_preprocessing import *
from utils import *
from models import *
from train import *
from evaluate import *

import tensorflow as tf
print(tf.config.list_physical_devices("GPU"))

set_seeds(RANDOM_SEED)


In [ ]:
df = load_and_prepare_data(DATA_PATH)
train, val, test = split_data(df)

feature_cols = [
    c for c in train.columns
    if c not in ["server_timestamp", TARGET_COLUMN]
]

train, val, test = scale_features(train, val, test, feature_cols)


In [ ]:
X_train, y_train = create_sequences(
    train[feature_cols].values,
    train[TARGET_COLUMN].values,
    SEQUENCE_LENGTH
)

X_val, y_val = create_sequences(
    val[feature_cols].values,
    val[TARGET_COLUMN].values,
    SEQUENCE_LENGTH
)

X_test, y_test = create_sequences(
    test[feature_cols].values,
    test[TARGET_COLUMN].values,
    SEQUENCE_LENGTH
)


In [ ]:
bounds = {
    "units": ([64, 128], "categorical"),
    "dropout": ([0.3, 0.6], "float"),
    "learning_rate": ([1e-4, 5e-4], "float"),
    "batch_size": ([32], "categorical")
}

optimizer = NinjaOptimizationAlgorithm(
    objective_function=lambda p: objective_function_lstm(
        p, X_train, y_train, X_val, y_val
    ),
    bounds=bounds,
    n_agents=N_AGENTS,
    max_iterations=MAX_ITERATIONS,
    exploration_factor=EXPLORATION_FACTOR,
    exploitation_factor=EXPLOITATION_FACTOR
)

best_params, _, _ = optimizer.optimize()


In [ ]:
final_model = create_lstm_model(
    best_params,
    X_train.shape[1],
    X_train.shape[2]
)

final_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=best_params["batch_size"],
    verbose=1
)

mae, rmse, r2, explanation = evaluate_model(
    final_model, X_test, y_test
)

print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)
print(explanation)
